# Define the task
The Feedback Task Datasets allow to combine multiple questions of different kinds, so the first step will be to define the aim of your project and the kind of data and feedback you will need to get there.

## Format records
A record in Argilla refers to a data item that requires annotation and can consist of one or multiple fields. For example, your records can include a pair of a prompt and an output. Currently, we only support plain text fields, but we plan to introduce support for markdown and images in the future.

Take some time to explore and find data that fits the purpose of your project. If you are planning to use public data, the [Datasets page](https://huggingface.co/datasets) of the Hugging Face Hub is a good place to start.

<div class="admonition hint">
Always check the licenses of the datasets to make sure you can legally use the dataset for your specfic use case.
</div>

Once you have a dataset, load it and inspect it to find the fields that you want to use in your Feedback dataset. A quick overview of the data will also help you formulate the right questions later.

In [ ]:
from datasets import load_dataset

dataset = load_dataset('databricks/databricks-dolly-15k', split='train')
dataset

In [ ]:
import pandas as pd

# turn it into a pandas dataframe to get a quick overview of a few examples
df = pd.DataFrame(dataset)
df

The next step is to create records following Argilla's Feedback Record format [link to Python reference].

The name of the fields will need to match the fields set up in the dataset configuration (see [Create and import a dataset](../practical_guides/create_and_import_dataset.ipynb)).

In [ ]:
# as we create the records, we can rename the fields and optionally filter the original dataset
records = [rg.FeedbackRecord(fields={"question": record["instruction"], "answer": record["response"]}) for record in dataset if record["category"]=="open_qa"]

## Define questions
To collect feedback for your dataset, you need to formulate questions. The Feedback Task currently supports the following types of questions:

- Rating: These questions require annotators to select one option from a list of integer values. This type is useful for collecting numerical scores.
- Text: These questions offer annotators a free-text area where they can enter any text. This type is useful for collecting natural language data, such as corrections or explanations.

<div class="admonition note">
We have plans to expand the range of supported question types in future releases of the Feedback Task.
</div>

You can define your questions using the Python SDK and set up the following configurations:

- `name`: A shortname for the question.
- `title`: The text displayed in the UI.
- `description` (optional): The text to be displayed in the question tooltip in the UI. You can use it to give more context or information to annotators.
- `required`: Set your question as required or optional. Annotators must answer all required questions to submit a response, but they have the choice to answer optional questions or not.
- `values`: In a RatingQuestion, these are the rating options represented as a list of integer values.

<div class="admonition note">
The order of the questions in the UI follows the order in which these are added to the dataset in the Python SDK.
</div>

In [ ]:
# list of questions to display in the feedback form
questions =[
    rg.RatingQuestion(
        name="rating", 
        title="Rate the quality of the response:", 
        description="1 = very bad - 5= very good",
        required=True,
        values=[1,2,3,4,5]
    ),
    rg.TextQuestion(
        name="corrected-text",
        title="Provide a correction to the response:",
        required=False
    )
]


## Write guidelines
Once you have decided on the data to show and the questions to ask, it's important to provide clear guidelines to the annotators. These guidelines help them understand the task and answer the questions consistently. You can provide guidelines in two ways:

- In the dataset guidelines: this is added as an argument when you create your dataset in the Python SDK (see below). It will appear in the dataset settings in the UI.
- As question descriptions: these are added as an argument when you create questions in the Python SDK (see above). This text will appear in a tooltip next to the question in the UI.

It is good practice to use at least the dataset guidelines, if not both methods. In the guidelines, you can include a description of the project, details on how to answer each question with examples, instructions on when to discard a record, etc. Question descriptions should be short and provide context to a specific question. They can be a summary of the guidelines to that question, but often times that is not sufficient to align the whole annotation team.